In [1]:
%matplotlib inline

In [2]:
import pymysql
import pandas as pd
import getpass
from textblob import TextBlob
import re

In [4]:
conn = pymysql.connect(host="mysql",
                       port=3306,user="jovyan",
                       passwd=getpass.getpass("Enter MySQL passwd for jovyan"),db='mimic2')
cursor = conn.cursor()

Enter MySQL passwd for jovyan········


### Use Pandas and SQL to create a dataframe with the following:
* subject_id
* hospital admission id
* text of the radiology report
* Limit the number of reports to 10000

In [5]:
rad_data = \
pd.read_sql("""SELECT noteevents.subject_id, 
                      noteevents.hadm_id,
                      noteevents.text 
               FROM noteevents
               WHERE noteevents.category = 'RADIOLOGY_REPORT' LIMIT 5000""",conn)
rad_data.head(5)

,subject_id,hadm_id,text
0,56,28766.0,\n\n\n DATE: [**2644-1-17**] 10:53 AM\n ...
1,56,28766.0,\n\n\n DATE: [**2644-1-17**] 10:53 AM\n ...
2,56,28766.0,\n\n\n DATE: [**2644-1-17**] 10:43 AM\n ...
3,56,28766.0,\n\n\n DATE: [**2644-1-17**] 6:37 AM\n ...
4,56,28766.0,\n\n\n DATE: [**2644-1-19**] 12:09 PM\n ...


In [6]:
rad_data.shape

(5000, 3)

In [7]:
print(rad_data.iloc[20]["text"])




     DATE: [**3352-7-6**] 10:20 AM
     CHEST (PORTABLE AP)                                             Clip # [**Clip Number (Radiology) 1631**]
     Reason: fever- eval for infection
     Admitting Diagnosis: NON-HODGKIN'S LYMPHOMIA\BONE MARROW TRANSPLANT
     ______________________________________________________________________________
     UNDERLYING MEDICAL CONDITION:
      54 year old man with T-cell lymphoma, neutropenia
     REASON FOR THIS EXAMINATION:
      fever- eval for infection
     ______________________________________________________________________________
                                     FINAL REPORT
     INDICATION:  Neutropenia and fever.
     
     COMPARISON: [**6-23**], [**6-6**] and [**3352-6-3**].
     
     CHEST AP PORTABLE RADIOGRAPH:  The cardiac, mediastinal and hilar contours are
     unremarkable.  The left lung is clear.  There is a somewhat well circumscribed
     opacity immediately overlying the catheter which runs along the lateral right
 

### Write a function that returns the impression section of a report

#### Hints

* Not every report will have an impression section
* "INTERPRETATION" and "CONCLUSIONS" might be synonyms for "IMPRESSION"
* If an impression section is not found, return the entire report.

In [24]:
def get_impression(report):
    for section in ["IMPRESSION:", "INTERPRETATION:", "CONCLUSIONS:"]:
        if section in report:
            return report[report.find(section):]
    return report
def get_impression(report ):
    for section in ["IMPRESSION:", "INTERPRETATION:", "CONCLUSIONS:"]:
        if section in report:
            return report.split(section)[1].strip()
    return report


In [25]:
get_impression2(rad_data.iloc[20]["text"])

'1)  There is a somewhat well-circumscribed but subtle opacity immediately\n     adjacent to the external portion of the catheter overlying the right upper\n     lobe.  This may be related to the catheter itself but it is difficult to\n     exclude an underlying opacity in this region.  Correlation with physical exam\n     recommended.'

#### In how many reports did we find an impression section?

In [32]:
def has_impression(report):
    for section in ["IMPRESSION:", "INTERPRETATION:", "CONCLUSIONS:"]:
        if section in report:
            return True
    return False    


In [33]:
sum([has_impression(report) for report in rad_data["text"]])

4043

### Define Regular expressions for data cleansing

* Write a regular expression to replace dates in the reports with ``[**DATE**]``
* Write a regular expression to replace times in the reports with ``[**TIME**]``
* Write a regular expression to replaces digits with "d", (e.g. "43 cc" would become "dd cm")

#### Hints

* Look at some sample reports to see what dates and times look like in the reports
* What order would you need to apply the regular expressions?

In [34]:
for i in range(10):
    print(rad_data.iloc[i]['text'])
    print("*"*42,"\n\n")




     DATE: [**2644-1-17**] 10:53 AM
     MR HEAD W & W/O CONTRAST; MR CONTRAST GADOLIN                   Clip # [**Clip Number (Radiology) 12569**]
     Reason: R ICB and HX brain mets - eval - also with DWI for CVA Do MR
      Contrast: MAGNEVIST Amt: 15
     ______________________________________________________________________________
     UNDERLYING MEDICAL CONDITION:
      [**Age over 90 **] year old woman with lung CA- mets to brain                                   
     REASON FOR THIS EXAMINATION:
      R ICB and HX brain mets - eval - also with DWI for CVA Do MRI both with and 
      without contast please
     ______________________________________________________________________________
                                     FINAL REPORT
     EXAMINATION:  MRI of the brain with and without gadolinium.
     
     INDICATION:  [**Age over 90 **] year old woman with lung cancer and right intracranial bleed
     and history of brain metastases.  Please evaluate for acute infar

In [35]:
date = re.compile(r"""\d""")

In [36]:
print(rad_data.iloc[0]["text"])




     DATE: [**2644-1-17**] 10:53 AM
     MR HEAD W & W/O CONTRAST; MR CONTRAST GADOLIN                   Clip # [**Clip Number (Radiology) 12569**]
     Reason: R ICB and HX brain mets - eval - also with DWI for CVA Do MR
      Contrast: MAGNEVIST Amt: 15
     ______________________________________________________________________________
     UNDERLYING MEDICAL CONDITION:
      [**Age over 90 **] year old woman with lung CA- mets to brain                                   
     REASON FOR THIS EXAMINATION:
      R ICB and HX brain mets - eval - also with DWI for CVA Do MRI both with and 
      without contast please
     ______________________________________________________________________________
                                     FINAL REPORT
     EXAMINATION:  MRI of the brain with and without gadolinium.
     
     INDICATION:  [**Age over 90 **] year old woman with lung cancer and right intracranial bleed
     and history of brain metastases.  Please evaluate for acute infar

In [37]:
print(rad_data.iloc[0]["text"])




     DATE: [**2644-1-17**] 10:53 AM
     MR HEAD W & W/O CONTRAST; MR CONTRAST GADOLIN                   Clip # [**Clip Number (Radiology) 12569**]
     Reason: R ICB and HX brain mets - eval - also with DWI for CVA Do MR
      Contrast: MAGNEVIST Amt: 15
     ______________________________________________________________________________
     UNDERLYING MEDICAL CONDITION:
      [**Age over 90 **] year old woman with lung CA- mets to brain                                   
     REASON FOR THIS EXAMINATION:
      R ICB and HX brain mets - eval - also with DWI for CVA Do MRI both with and 
      without contast please
     ______________________________________________________________________________
                                     FINAL REPORT
     EXAMINATION:  MRI of the brain with and without gadolinium.
     
     INDICATION:  [**Age over 90 **] year old woman with lung cancer and right intracranial bleed
     and history of brain metastases.  Please evaluate for acute infar

In [38]:
digits = re.compile(r"""\d""")

In [39]:
print(digits.sub("d", rad_data.iloc[0]["text"]))




     DATE: [**dddd-d-dd**] dd:dd AM
     MR HEAD W & W/O CONTRAST; MR CONTRAST GADOLIN                   Clip # [**Clip Number (Radiology) ddddd**]
     Reason: R ICB and HX brain mets - eval - also with DWI for CVA Do MR
      Contrast: MAGNEVIST Amt: dd
     ______________________________________________________________________________
     UNDERLYING MEDICAL CONDITION:
      [**Age over dd **] year old woman with lung CA- mets to brain                                   
     REASON FOR THIS EXAMINATION:
      R ICB and HX brain mets - eval - also with DWI for CVA Do MRI both with and 
      without contast please
     ______________________________________________________________________________
                                     FINAL REPORT
     EXAMINATION:  MRI of the brain with and without gadolinium.
     
     INDICATION:  [**Age over dd **] year old woman with lung cancer and right intracranial bleed
     and history of brain metastases.  Please evaluate for acute infar

In [40]:
rad_data["impression"] = \
rad_data.apply(lambda row: digits.sub("d", get_impression(row["text"])).lower(), axis=1)

In [41]:
rad_data.head()

,subject_id,hadm_id,text,impression
0,56,28766.0,\n\n\n DATE: [**2644-1-17**] 10:53 AM\n ...,\n\n\n date: [**dddd-d-dd**] dd:dd am\n ...
1,56,28766.0,\n\n\n DATE: [**2644-1-17**] 10:53 AM\n ...,\n\n\n date: [**dddd-d-dd**] dd:dd am\n ...
2,56,28766.0,\n\n\n DATE: [**2644-1-17**] 10:43 AM\n ...,stable appearance of right parietal lobe and l...
3,56,28766.0,\n\n\n DATE: [**2644-1-17**] 6:37 AM\n ...,cardiomegaly and mild chf. nasogastric tube a...
4,56,28766.0,\n\n\n DATE: [**2644-1-19**] 12:09 PM\n ...,marked improvement in left perihilar alveolar ...


In [43]:
rad_data.head()

,subject_id,hadm_id,text,impression
0,56,28766.0,\n\n\n DATE: [**2644-1-17**] 10:53 AM\n ...,\n\n\n date: [**dddd-d-dd**] dd:dd am\n ...
1,56,28766.0,\n\n\n DATE: [**2644-1-17**] 10:53 AM\n ...,\n\n\n date: [**dddd-d-dd**] dd:dd am\n ...
2,56,28766.0,\n\n\n DATE: [**2644-1-17**] 10:43 AM\n ...,stable appearance of right parietal lobe and l...
3,56,28766.0,\n\n\n DATE: [**2644-1-17**] 6:37 AM\n ...,cardiomegaly and mild chf. nasogastric tube a...
4,56,28766.0,\n\n\n DATE: [**2644-1-19**] 12:09 PM\n ...,marked improvement in left perihilar alveolar ...


In [42]:
print(rad_data.iloc[20]["impression"])

d)  there is a somewhat well-circumscribed but subtle opacity immediately
     adjacent to the external portion of the catheter overlying the right upper
     lobe.  this may be related to the catheter itself but it is difficult to
     exclude an underlying opacity in this region.  correlation with physical exam
     recommended.


### How many unique words occur in the corpus?

#### Hint

1. Use TextBlob
1. Put all the reports into a single string


In [50]:
reports = " ".join([txt for txt in rad_data["impression"]])
len(reports)
uwords = set(TextBlob(reports.lower()).words)

In [51]:
len(uwords)


6396

### Are all words equal?

In [52]:
from gensim.parsing.preprocessing import STOPWORDS
STOPWORDS

frozenset({'a',
           'about',
           'above',
           'across',
           'after',
           'afterwards',
           'again',
           'against',
           'all',
           'almost',
           'alone',
           'along',
           'already',
           'also',
           'although',
           'always',
           'am',
           'among',
           'amongst',
           'amoungst',
           'amount',
           'an',
           'and',
           'another',
           'any',
           'anyhow',
           'anyone',
           'anything',
           'anyway',
           'anywhere',
           'are',
           'around',
           'as',
           'at',
           'back',
           'be',
           'became',
           'because',
           'become',
           'becomes',
           'becoming',
           'been',
           'before',
           'beforehand',
           'behind',
           'being',
           'below',
           'beside',
           'besides'

### Should STOPWORDS be the same across domains?

In [53]:
my_stop_words = frozenset(["a", "am", "an", "and", "are", "as", "at", "be", "for", "is", "the", "is", "of", "which", ])

## Create a single string with all the reports

#### Hints, etc.
* Use List Comprehension
* Use string joins
* Iterate over the rows of the data frame

In [55]:
no_stop_impression = set([word for word in TextBlob(reports.lower()).words 
                          if word not in my_stop_words])

In [56]:
len(no_stop_impression)

6383

In [57]:
no_stop_impression

{'related',
 'ring',
 'noncontrast',
 'entirely',
 'reconstruction',
 'cystoscopic',
 'dopp',
 'well-circumscribed',
 'collateral',
 'tailored',
 'x-rays',
 'appendix',
 'shoulders',
 'colostomies',
 'lvier',
 'doing',
 'convex',
 'bronchopulmonary',
 'low',
 'used',
 'pcp',
 'erythropietic',
 "it's",
 'th',
 'sub',
 'necrotizing',
 'radius',
 'duodenum',
 'differentiate',
 'pv',
 'conjunction',
 'calcification',
 'progressing',
 'fall',
 'white-out',
 'bleed',
 'similiar',
 'thrombocytopenia',
 'broncial',
 'non-specifcic',
 'ddrd',
 'anterocutaneous',
 'then',
 'csru',
 'pericardium',
 'fusion',
 'persisting',
 'pulm',
 'maximum',
 'inspissated',
 'gaseous',
 'hyperemia',
 'contacted',
 'current',
 'water-soluble',
 'suggested',
 'facial',
 'cardiopulmonary',
 'review',
 'walls',
 'pseudoaneurys',
 'medial',
 'bronchus',
 'effusions/pleural',
 'therapist',
 'incisional',
 'icd',
 'acceleration',
 'cri',
 'cavernous',
 'malposition',
 'avr/mvr/tv',
 'dictate',
 'simple',
 'intraperit'

### Define a vector space for the radiology corpus

#### Hints

1. How would you build a corpus from words only occuring more than N times?

In [59]:
from collections import Counter
word_counts = Counter([word for word in TextBlob(reports.lower()).words 
                          if word not in my_stop_words])
word_counts

Counter({'date': 1536,
         'dddd-d-dd': 1149,
         'dd': 4525,
         'mr': 283,
         'head': 212,
         'w': 318,
         'w/o': 569,
         'contrast': 1932,
         'gadolin': 55,
         'clip': 2756,
         'number': 1469,
         'radiology': 1535,
         'ddddd': 931,
         'reason': 2170,
         'r': 82,
         'icb': 4,
         'hx': 29,
         'brain': 188,
         'mets': 32,
         'eval': 318,
         'also': 258,
         'with': 4748,
         'dwi': 7,
         'cva': 16,
         'do': 55,
         'magnevist': 85,
         'amt': 550,
         'underlying': 821,
         'medical': 736,
         'condition': 715,
         'age': 33,
         'over': 860,
         'year': 664,
         'old': 696,
         'woman': 280,
         'lung': 745,
         'ca': 58,
         'to': 3389,
         'this': 1717,
         'examination': 975,
         'mri': 130,
         'both': 295,
         'without': 391,
         'contast': 2,
      

In [62]:
word_counts.most_common(6000)[-100:]

[('aortoiliac', 1),
 ('tenosynovitis', 1),
 ('flexor', 1),
 ('hallucis', 1),
 ('longus', 1),
 ('tendon', 1),
 ('consciousness', 1),
 ('flow-limiting', 1),
 ('fem-akpop', 1),
 ('created', 1),
 ('ankle-brachial', 1),
 ('femoropopliteal', 1),
 ('aspirating', 1),
 ('conducted', 1),
 ('puree', 1),
 ('cookie', 1),
 ('pill', 1),
 ('coated', 1),
 ('cup', 1),
 ('responds', 1),
 ('effective', 1),
 ('ingestion', 1),
 ('videoswallow', 1),
 ('adajcent', 1),
 ('diagnoses', 1),
 ('originating', 1),
 ('insufficience', 1),
 ('excluding', 1),
 ('followupl', 1),
 ('versusimited', 1),
 ('emesis', 1),
 ('gas-distended', 1),
 ('pseudo-obstruction', 1),
 ('hepatology', 1),
 ('precise', 1),
 ('thoracoabdominal', 1),
 ('gastroduodenal', 1),
 ('intstrument', 1),
 ('offf', 1),
 ('bms', 1),
 ('removede', 1),
 ('hemothoraces', 1),
 ('buttress', 1),
 ('transfixing', 1),
 ('midshaft', 1),
 ('washer', 1),
 ('contiguously', 1),
 ('quadrigeminal', 1),
 ('longitudinally', 1),
 ('supposition', 1),
 ('prominently', 1),
 (

### Create a new column named `"impression no stops"` where [stop words](https://en.wikipedia.org/wiki/Stop_words) have been dropped from the impression

* The gensim package has stop words defined (``from gensim.parsing.preprocessing import STOPWORDS``

#### Hints
1. Do you agree with dropping all the stop words?
1. How could we create a new stopwords frozen set absent the terms we wan't to keep (double negative?)
1. You could use a regular expressions substitution or token the report first and operate on the list of words.

In [ ]:
rad_data["impression no stops"] = None

In [ ]:
rad_data.iloc[0]["impression"]

In [ ]:
rad_data.iloc[0]["impression no stops"]

In [ ]:
unique_impression_words = set(TextBlob(" ".join(rad_data["impression no stops"])).words)


In [ ]:
word_map = dict(zip(unique_impression_words,range(len(unique_impression_words))))

In [ ]:
len(word_map)

In [ ]:
word_map